## Parse SAPhon inventory spreadsheets to look for errors

Use this notebook to process a `.tsv` file downloaded from the SAPhon [Tupian Nasal Typology Input](https://docs.google.com/spreadsheets/d/1dvXFvLIV4y84CglgjAl-ZVb09IuGazs1SzFO_UJpmnI/edit#gid=1164878023) spreadsheet. Warning messages are emitted for spreadsheet values that do not validate. Make corrections in the spreadsheet, then download a new `.tsv` file and try again.

To run the cells in this notebook, click on the cell to execute, then on the `Run` button or press `Ctrl`+`Enter` on the keyboard. Execute cells in order from top to bottom.

## Getting started

When you are ready to validate data from the [input spreadsheet](https://docs.google.com/spreadsheets/d/1dvXFvLIV4y84CglgjAl-ZVb09IuGazs1SzFO_UJpmnI/edit#gid=1164878023) for a language:

1. Select the tab for the language you want to check by setting the `idx` variable below.
1. Run the following cells to perform the validation tests.

In [ ]:
import spreadsheet
import requests
import pandas as pd
from pathlib import Path
cachedir = Path.home() / 'Downloads'

## Get Tupian input spreadsheet lang tabs

Collect the language tabs from the input spreadsheet into a dataframe, one row per lang tab.

In [ ]:
ssdf = pd.DataFrame.from_records(list(spreadsheet.langsheets.values()))
ssdf['tabname'] = list(spreadsheet.langsheets.keys())
ssdf['yaml'] = ssdf['short'] + '.yaml'
assert(~ssdf['gid'].duplicated().any())
assert(~ssdf['tabname'].duplicated().any())
ssdf

## Read a language file

Choose a language tab index number <= `66`. The next cell reads the file into a dictionary named `lang`. The dictionary contains the content of the `synthesis` and `ref` documents.

In [ ]:
idx = 6
row = ssdf[ssdf['include']].iloc[idx]
r = requests.get(f"{spreadsheet.puburl}/pub?gid={row['gid']}&single=true&output=tsv")
r.encoding = 'utf-8'
cachedtsv = cachedir / f"{row['short']}.tsv"
with open(cachedtsv, 'w', encoding='utf-8') as out:
    out.write(r.text)
    print(f'Wrote lang idx {idx} to {cachedtsv}')
lang = spreadsheet.read_lang(cachedtsv)

Check the `Natural Classes`/`Segments` fields. The return value `natclasses` is a dictionary that maps each `synthesis`/`ref`document to its structured natural class data that can be referenced in the processes of that document. The `flatnatclasses` variable is a dictionary that contains a simple list of all the symbols used for natural class data for each document.

In [ ]:
natclasses, flatnatclasses, catsymb = spreadsheet.check_natclasses(lang)

Check the `allophones` field for errors. The target phone for each allophone must match one of the natural class symbols.

In [ ]:
allophones, alloprocs = spreadsheet.check_allophones(lang, flatnatclasses)

Validate the `Morpheme IDs` field. The return value `morph_id_map` is a dictionary that maps each `synthesis`/`ref`document to a set of morpheme ids that can be referenced in the processes of that document.

In [ ]:
morph_id_map = spreadsheet.check_morpheme_ids(lang)

Check the processes of the language.

In [ ]:
spreadsheet.check_procs(lang, flatnatclasses, morph_id_map, catsymb, alloprocs)